In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score

# Load the train data
train_data = pd.read_csv('train.csv')

# Replace missing values with mean for numerical columns
numerical_cols = train_data.select_dtypes(include=['float64']).columns
train_data[numerical_cols] = train_data[numerical_cols].fillna(train_data[numerical_cols].mean())

# Replace missing values with mode for categorical columns
categorical_cols = train_data.select_dtypes(include=['object']).columns
for col in categorical_cols:
    train_data[col] = train_data[col].fillna(train_data[col].mode()[0])

# Label encoding for categorical columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    label_encoders[col] = le

# Splitting the dataset into features and target
X = train_data.drop(['ID', 'Target'], axis=1)
y = train_data['Target']

# Define the deep learning model
def create_model(learning_rate=0.001, dropout_rate=0.5, regularization_strength=0.01):
    model = Sequential([
        Dense(256, input_shape=(X.shape[1],), activation='relu'),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(regularization_strength)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(regularization_strength)),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Define grid search parameters
param_grid = {
    'learning_rate': [0.001, 0.0005, 0.0001],
    'dropout_rate': [0.4, 0.5, 0.6],
    'regularization_strength': [0.01, 0.001, 0.0001],
    'batch_size': [32, 64, 128]
}

In [12]:
best_score = 0
best_params = {}
for lr in param_grid['learning_rate']:
    for dr in param_grid['dropout_rate']:
        for reg in param_grid['regularization_strength']:
            for batch in param_grid['batch_size']:
                model = create_model(learning_rate=lr, dropout_rate=dr, regularization_strength=reg)
                history = model.fit(X, y, epochs=100, batch_size=batch, verbose=0)
                
                # Calculate F1 score on validation set (You can also use cross-validation here)
                X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
                y_val_pred = (model.predict(X_val) > 0.5).astype(int).flatten()
                f1 = f1_score(y_val, y_val_pred)
                
                if f1 > best_score:
                    best_score = f1
                    best_params = {'learning_rate': lr, 'dropout_rate': dr, 'regularization_strength': reg, 'batch_size': batch}

# Print results
print(f"Best F1 Score: {best_score} using {best_params}")

# Use the best parameters obtained from grid search
final_model = create_model(learning_rate=best_params['learning_rate'], 
                           dropout_rate=best_params['dropout_rate'], 
                           regularization_strength=best_params['regularization_strength'])

# Train the final model
final_model.fit(X, y, epochs=100, batch_size=best_params['batch_size'], verbose=1)

# Predictions on test set
test_data = pd.read_csv('test.csv')

# Replace missing values with mean for numerical columns in test data
test_data[numerical_cols] = test_data[numerical_cols].fillna(test_data[numerical_cols].mean())

# Replace missing values with mode for categorical columns in test data
for col in categorical_cols:
    test_data[col] = test_data[col].fillna(test_data[col].mode()[0])

# Label encoding for categorical columns in test data
for col in categorical_cols:
    test_data[col] = label_encoders[col].transform(test_data[col])

test_predictions = (final_model.predict(test_data.drop('ID', axis=1)) > 0.5).astype(int).flatten()

# Create submission file
submission = pd.DataFrame({'ID': test_data['ID'], 'Target': test_predictions})
submission.to_csv('submission.csv', index=False)

c:\Users\SARVESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


c:\Users\SARVESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


c:\Users\SARVESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step


c:\Users\SARVESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step
Best F1 Score: 0.7466307277628033 using {'learning_rate': 0.0005, 'dropout_rate': 0.4, 'regularization_strength': 0.001, 'batch_size': 64}


c:\Users\SARVESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - accuracy: 0.5409 - loss: 1.1853
Epoch 2/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5270 - loss: 1.1010
Epoch 3/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5414 - loss: 1.0158
Epoch 4/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5294 - loss: 1.0103
Epoch 5/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5152 - loss: 0.9903
Epoch 6/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5373 - loss: 0.9389
Epoch 7/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5452 - loss: 0.9396
Epoch 8/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5487 - loss: 0.9178
Epoch 9/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5520 - loss: 0.9036
Epoch 10/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5399 - loss: 0.8937
Epoch 11/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5397 - loss: 0.8899
Epoch 12/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/ste